In [ ]:
import pandas as pd
from langchain_community.llms import Ollama
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

df = pd.read_csv('test2.csv')

# LLM 모델 초기화
llm = Ollama(model='gemma2')

# JSONOutputParser 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿 설정
prompt_template = ChatPromptTemplate.from_messages([
    ("system",
     '''당신은 주어진 뉴스 기사 제목을 참고하여 새로운 제목을 만드는 어시스턴트입니다.
     아래 규칙을 반드시 따라주세요:
     
     1. 입력으로 주어진 뉴스 제목과 비슷한 맥락이지만, 다른 표현으로 바꾼 새로운 제목을 하나만 생성해주세요.
     2. 원본 제목에 나오는 중요한 고유명사, 핵심 키워드는 유지해주세요.
     3. 출력은 반드시 아래 JSON 형식을 지켜주세요:
        {{ "re_text": "생성된 제목" }}
     4. 마크다운 강조 표시(**) 등은 사용하지 마세요.
     
     입력 예시 제목:
     {example}
     '''),
    ("user", "{req}")
])

# 템플릿에 고정 지시사항 주입
prompt_template = prompt_template.partial(
    req="위 예시 제목과 비슷하지만 새로운 표현의 제목을 JSON 형식으로 생성해주세요."
)

# 빈 리스트를 준비하여 새로운 제목을 저장
new_titles = []

# 각 row에 대해 새로운 제목 생성
for index, row in df.iterrows():
    example_title = row['text']  # 예시로 사용할 제목 (text 열의 내용)
    
    # 프롬프트 템플릿에 제목을 주입하여 프롬프트 생성
    prompt = prompt_template.partial(example=example_title)
    
    # 체인을 통해 LLM 실행
    response = (prompt | llm).invoke(input={})  # 빈 딕셔너리로 입력 전달
    
    # JSON 파싱 후 새로운 제목 추출
    new_title = parser.parse(response)["re_text"]
    
    # 새로운 제목을 리스트에 추가
    new_titles.append(new_title)

# 새로운 칼럼 're_text'를 데이터프레임에 추가
df['re_text'] = new_titles

In [ ]:
df.to_csv('not_noisy_aug.csv')